# Libraries Used :

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

### Load the data into Pandas dataframe

In [2]:
df = pd.read_csv('../input/olidtraining/olid-training-v1.0.tsv',sep='\t')
df.head()

In [3]:
df["tweet"][1]

In [4]:
df.info()

Understand the distribution better.

In [5]:
sns.countplot(df["subtask_a"])
plt.xlabel('Label')
plt.title('Number of offensive and non-offensive tweets')

* Create input and output vectors.
* Process the labels.

In [6]:
X = df["tweet"]
Y = df["subtask_a"]
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

Spliting the dataset into training and test data.

In [7]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)

### Pre process data
* Tokenizing and padding 
* There are many ways of taking the *max_len* and here an arbitrary length of 150 is chosen.

In [8]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

### RNN with LSTM


In [9]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model
    

In [10]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Fitting the datset.

In [11]:
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=5,
          validation_split=0.2)

Processing the test dataset.

In [12]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

Evaluate the model on the test set.

In [13]:
accr = model.evaluate(test_sequences_matrix,Y_test)

In [14]:
print('Evaluating the model on the test dataset\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

In [15]:
p_lstm = (model.predict(test_sequences_matrix) > 0.5).astype("int32")
mtx = confusion_matrix(Y_test, p_lstm)
sns.heatmap(mtx, annot=True, fmt='d', linewidths=.5, 
               cmap="Blues", cbar=False)
plt.ylabel('True label')
plt.xlabel('Predicted label')

In [17]:
f1 = f1_score(Y_test, p_lstm, average= "macro")
f1

In [18]:
max_words1 = 100
max_len1 = 150
text_sequences = tok.texts_to_sequences(["#HappyBdayPMModi One may hate him. But u should agree that he is the most hard working PM India has ever witnessed.😊 URL"])
text_sequences_matrix = sequence.pad_sequences(text_sequences,maxlen=max_len1)
text_sequences_matrix

In [19]:
model.predict(text_sequences_matrix)

In [20]:
output_lstm = (model.predict(text_sequences_matrix) > 0.5).astype("int32")
output_lstm